In [1]:
%pip install faker


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from faker import Faker
import random

In [ ]:
# Load your dataset
df = pd.read_csv('original_data.csv')

# Initialize Faker
fake = Faker()

# Function to determine if a column is normally distributed
def is_normally_distributed(column):
    stat, p = stats.shapiro(column.dropna())
    return p > 0.05  # If p-value > 0.05, assume normal distribution

# Analyzing and generating the synthetic dataset
synthetic_data = pd.DataFrame()

for column in df.columns:
    if df[column].dtype == 'float64' or df[column].dtype == 'int64':
        # Check if the column is normally distributed
        if is_normally_distributed(df[column]):
            mean = df[column].mean()
            std = df[column].std()
            synthetic_data[column] = np.random.normal(mean, std, df.shape[0])
        else:
            # Use a different method if not normally distributed
            synthetic_data[column] = np.random.choice(df[column].dropna(), size=df.shape[0])
    elif df[column].dtype == 'object':
        # Ensuring the same frequency distribution for categorical data
        categories, frequencies = np.unique(df[column].dropna(), return_counts=True)
        probabilities = frequencies / frequencies.sum()
        synthetic_data[column] = np.random.choice(categories, size=df.shape[0], p=probabilities)
    elif pd.api.types.is_datetime64_any_dtype(df[column]):
        # Dealing with dates
        start_date = df[column].min()
        end_date = df[column].max()
        synthetic_data[column] = [fake.date_between(start_date=start_date, end_date=end_date) for _ in range(df.shape[0])]
    else:
        # Replicate other types as they are
        synthetic_data[column] = df[column]

# Save the synthetic dataset
synthetic_data.to_csv('synthetic_dataset.csv', index=False)
